# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit North and hold:
hand = '5.AK93.AKJT72.T4'

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# the auction goes:
auction = ["PAD_START","PASS", "PASS", "2D"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=0, dealer=1, ddsolver=dds, bba_is_controlling=False, verbose=False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': 'PASS', 'insta_score': 0.2, 'adjustment': 0.48, 'expected_score': 193, 'expected_imp': 5.02, 'expected_tricks': 5.0, 'alert': 'False', 'who': 'BBA', 'explanation': ' -- ; 16- HCP'}, {'call': 'X', 'insta_score': 0.113, 'adjustment': 0.27, 'expected_score': 17, 'expected_imp': -0.67, 'expected_tricks': 7.7}, {'call': '3D', 'insta_score': 0.816, 'adjustment': 1.96, 'expected_score': -125, 'expected_imp': -4.35, 'expected_tricks': 7.6}]


In [4]:
bid.samples

['x.AKxx.AKJTxx.Tx Jxxxx.QTxx..KQJx AKTxx.Jxx.xx.xxx Qx.xx.Qxxxx.Axxx - 0.97150 | P-P-2D-3D-P-4S-P-P-P (7) | P-P-2D-X-P-2S-P-P-P (7) | P-P-2D-P-P-P (4)',
 'x.AKxx.AKJTxx.Tx Axxxx.Txxx..Axxx QJTx.QJx.xx.QJxx Kxx.xx.Qxxxx.Kxx - 0.96830 | P-P-2D-3D-P-4S-P-P-P (6) | P-P-2D-X-P-2S-P-P-P (6) | P-P-2D-P-P-P (4)',
 'x.AKxx.AKJTxx.Tx AKxx.Jxxx..xxxxx Jxxxx.Tx.xx.AKJx QTx.Qxx.Qxxxx.Qx - 0.96338 | P-P-2D-3D-P-4S-P-P-P (7) | P-P-2D-X-P-2S-P-P-P (7) | P-P-2D-P-P-P (2)',
 'x.AKxx.AKJTxx.Tx KQxxx.QJx.xx.Jxx ATxxx.xxxx..Kxxx Jx.Tx.Qxxxx.AQxx - 0.96309 | P-P-2D-3D-P-4S-P-P-P (7) | P-P-2D-X-P-2S-P-P-P (7) | P-P-2D-P-P-P (4)',
 'x.AKxx.AKJTxx.Tx Kxxx.xxx.x.KQJxx QJxxx.JTx.x.Axxx ATx.Qxx.Qxxxx.xx - 0.96230 | P-P-2D-3D-P-4S-P-P-P (8) | P-P-2D-X-P-2S-P-P-P (8) | P-P-2D-P-P-P (3)',
 'x.AKxx.AKJTxx.Tx QTxx.Jxx.xx.AQxx AJxxx.Qxx..Jxxxx Kxx.Txx.Qxxxx.Kx - 0.96118 | P-P-2D-3D-P-4S-P-P-P (8) | P-P-2D-X-P-2S-P-P-P (8) | P-P-2D-P-P-P (3)',
 'x.AKxx.AKJTxx.Tx JTxx.xxx.x.AKJxx KQxxx.JTx.x.Qxxx Axx.Qxx.Qxxxx.xx - 0.96

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START","PASS", "PASS", "2D"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 1, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 2, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 3, :], models.n_cards_bidding)}')

Good quality: 0.825164198615145
x.AKxx.AKJTxx.Tx Jxxxx.QTxx..KQJx AKTxx.Jxx.xx.xxx Qx.xx.Qxxxx.Axxx
x.AKxx.AKJTxx.Tx Axxxx.Txxx..Axxx QJTx.QJx.xx.QJxx Kxx.xx.Qxxxx.Kxx
x.AKxx.AKJTxx.Tx AKxx.Jxxx..xxxxx Jxxxx.Tx.xx.AKJx QTx.Qxx.Qxxxx.Qx
x.AKxx.AKJTxx.Tx KQxxx.QJx.xx.Jxx ATxxx.xxxx..Kxxx Jx.Tx.Qxxxx.AQxx
x.AKxx.AKJTxx.Tx Kxxx.xxx.x.KQJxx QJxxx.JTx.x.Axxx ATx.Qxx.Qxxxx.xx
x.AKxx.AKJTxx.Tx QTxx.Jxx.xx.AQxx AJxxx.Qxx..Jxxxx Kxx.Txx.Qxxxx.Kx
x.AKxx.AKJTxx.Tx JTxx.xxx.x.AKJxx KQxxx.JTx.x.Qxxx Axx.Qxx.Qxxxx.xx
x.AKxx.AKJTxx.Tx AKJTx.Jxx.x.xxxx Qxxxx.Txx.x.AJxx xx.Qxx.Qxxxx.KQx
x.AKxx.AKJTxx.Tx xxxx.Qxxxx.x.AQJ AJTxx.Jx.x.Kxxxx KQx.Tx.Qxxxx.xxx
x.AKxx.AKJTxx.Tx AKTx.JTxx.xx.Qxx QJxxx.Qxxx..Kxxx xxx.x.Qxxxx.AJxx
x.AKxx.AKJTxx.Tx Axxx.QJxxx.x.xxx QJTxx.Txx.x.AQxx Kxx.x.Qxxxx.KJxx
x.AKxx.AKJTxx.Tx KQxxx.Txxx..KJxx AJTxx.QJxx.xx.xx xx.x.Qxxxx.AQxxx
x.AKxx.AKJTxx.Tx AKxx.xx.x.Jxxxxx QTxxx.JTxx.x.AQx Jxx.Qxx.Qxxxx.Kx
x.AKxx.AKJTxx.Tx QTxxx.Jxx..AJxxx Axxx.QTxxx.xx.Qx KJx.x.Qxxxx.Kxxx
x.AKxx.AKJTxx.Tx